
#**Trabalho Prático 4**
##Bárbara  Faria (A85774) , Bruna Araújo (A84408) e Tiago Lima (A85126)
> Todos os problemas deste trabalho devem ser resolvidos usando pySMT   e  SMT’s  que suportem BitVec

In [ ]:
!pip install z3-solver

from z3 import *

def prove(f):
    s = Solver()
    s.add(Not(f))
    if s.check() == sat:
        print("A fórmula não é válida...")
    else:
        print("A fórmula é válida!")

#Enunciado

Considere o seguinte programa, em Python anotado, para multiplicação de dois inteiros de precisão limitada a 16 bits.

```python
assume (m>=0 and n>=0 and r==0 and x==m and y==n)
while y > 0:
  if y&1==1: 
    y, r  = y-1, r+x
  x, y = x<<1, y>>1
assert (r==m*n)
```

#Exercícios

**1.** Prove por indução a terminação deste programa

**2.** Pretende-se verificar a correção total deste  programa usando a metodologia dos invariantes e a metodologia do “single assignment unfolding”. Para isso,

**a.** Codifique usando a LPA (linguagem de programas anotadas) a forma recursiva deste programa. 

O programa pode ser escrito como um triplo de Hoare 
$\{\phi\} {\sf while} \; b \;{\sf do } \;C \{\psi\}$, onde $\phi$ é a pré-condição, $\psi$ é a pós-condição e $b$ é a condição do ciclo.

Uma vez que vamos verificar a correção do programa através da metodologia dos invariantes, basta garantir a validade dos seguintes triplos:

$\{\phi\}{\sf skip}\{\theta\}$, que corresponde à "inicialização";

$\{\theta\wedge b\}C\{\theta\}$, que corresponde à "preservação";

$\{\theta\wedge\neg b\}{\sf skip}\{\psi\}$, que corresponde à "utilidade", 

onde $\theta$ é o invariante.

Se forem válidos, então o triplo que está na conclusão é válido.

Inicialização:

LPA:

assume $\phi$ ; skip ; assert $\theta$

VC:

[assume $\phi$ ; skip ; assert $\theta$]

= $\phi$ $\to$ [skip ; assert $\theta$]

= $\phi$ $\to$ [assert $\theta$]

= $\phi$ $\to$ $\theta$

Preservação:

LPA:

assume ($\theta \wedge b$) ; (assume (y&1==1) ; y=y-1 ; r=r+x || assume (y&1!=1)) ; x=x<<1 ; y=y>>1 ; assert $\theta$

VC:

[assume ($\theta \wedge b$) ; (assume (y&1==1) ; y=y-1 ; r=r+x || assume (y&1!=1)) ; x=x<<1 ; y=y>>1 ; assert $\theta$]

= ($\theta \wedge b) \to$ [(assume (y&1==1) ; y=y-1 ; r=r+x || assume (y&1!=1)) ; x=x<<1 ; y=y>>1 ; assert $\theta$]

= ($\theta \wedge b) \to$ [(assume (y&1==1) ; y=y-1 ; r=r+x ; x=x<<1 ; y=y>>1 ; assert $\theta$) || (assume (y&1!=1) ; x=x<<1 ; y=y>>1 ; assert $\theta$)]

= ($\theta \wedge b) \to$ ([assume (y&1==1) ; y=y-1 ; r=r+x ; x=x<<1 ; y=y>>1 ; assert $\theta$] $\wedge$ [assume (y&1!=1) ; x=x<<1 ; y=y>>1 ; assert $\theta$])

= ($\theta \wedge b) \to$ (((y&1==1) $\to$ [y=y-1 ; r=r+x ; x=x<<1 ; y=y>>1 ; assert $\theta$]) $\wedge$ ((y&1!=1) $\to$ [x=x<<1 ; y=y>>1 ; assert $\theta$]))

= ($\theta \wedge b) \to$ (((y&1==1) $\to \theta$ [y/y>>1][x/x<<1][r/r+x][y/y-1]) $\wedge$ ((y&1!=1) $\to \theta$ [y/y>>1] [x/x<<1]))

Utilidade:

LPA:

assume ($\theta \wedge \neg b$) ; skip ; assert $\psi$

VC:

[assume ($\theta \wedge \neg b$) ; skip ; assert $\psi$]

= ($\theta \wedge \neg b$) $\to$ [skip ; assert $\psi$]

= ($\theta \wedge \neg b$) $\to$ [assert $\psi$]

= ($\theta \wedge \neg b$) $\to$ $\psi$

**b.** Proponha o invariante mais fraco que assegure a correção, codifique-o em SMT e prove a correção.

Vamos determinar um invariante;

Seja m==7, n==9, r==0, x==m, y==n e i a iteração do ciclo:

| i | m | n | r  | x   | y |
|---|---|---|----|-----|---|
| 0 | 7 | 9 | 0  | 7   | 9 |
| 1 | 7 | 9 | 7  | 14  | 4 |
| 2 | 7 | 9 | 7  | 28  | 2 |
| 3 | 7 | 9 | 7  | 56  | 1 |
| 4 | 7 | 9 | 63 | 112 | 0 |

Observando a tabela, concluímos que em cada iteração r+x*y = m*n

Adicionando a condição do ciclo, obtemos o nosso invariante (em SMT):

$\theta$ = And(y >= 0, r+x*y==m*n)

In [ ]:
m, n, r, x, y = BitVecs('m n r x y', 16)

phi = And(m>=0, n>=0, r==0, x==m, y==n)
theta = And(y>=0, r+x*y==m*n)
b = y>0
psi = r==m*n

sub1 = substitute(substitute(substitute(substitute(theta,(y,y>>1)),(x,x<<1)),(r,r+x)),(y,y-1))
sub2 = substitute(substitute(theta,(y,y>>1)),(x,x<<1))

inic = Implies(phi, theta)
pres = Implies(And(theta, b), And(Implies(y&1==1, sub1), Implies(y&1!=1, sub2)))
util = Implies(And(theta, Not(b)), psi)

print("\nInicialização:")
prove(inic)

print("\nPreservação:")
prove(pres)

print("\nUtilidade:")
prove(util)


Inicialização:
A fórmula é válida!

Preservação:
A fórmula é válida!

Utilidade:
A fórmula é válida!


**c.** Construa a definição iterativa do “single assignment unfolding”  usando um parâmetro limite $\,N\,$ e aumentando a pré-condição com a condição $$(n < N)\,\land\,(m<N)$$
O número de iterações vai ser controlado por este parâmetro $\,N\,$

I. Vamos começar por desenrolar o ciclo duas vezes:

```python
assume (m>=0 and n>=0 and r==0 and x==m and y==n)
if (y>0):
  if (y&1==1): 
    y=y-1 ; r=r+x
  x=x<<1 ; y=y>>1
  if (y > 0):
    if (y&1==1): 
      y=y-1 ; r=r+x
    x=x<<1 ; y=y>>1
    assume (y<=0)
assert (r==m*n)
```

II. Convertemos o programa ao formato "single assignment" (SA):

```python
assume (m0>=0 and n0>=0 and r0==0 and x0==m0 and y0==n0)
if (y0>0):
  if (y0&1==1): 
    y1=y0-1 ; r1=r0+x0
  else:
    x0=x0 ; y1=y0 ; r1=r0 ;
  x1=x0<<1 ; y2=y1>>1
  if (y2 > 0):
    if (y2&1==1): 
      y3=y2-1 ; r2=r1+x1
    else:
      x1=x1 ; y3=y2 ; r2=r1 ;
    x2=x1<<1 ; y4=y3>>1
    assume (y4<=0)
  else:
    x2=x1 ; y4=y2 ; r2=r1 ;
else:
  x2=x0 ; y4=y0 ; r2=r0 ;
assert (r2==m0*n0)
```

III. Vamos agora traduzir o programa para linguagem de fluxos:

```python
assume (m0>=0 and n0>=0 and r0==0 and x0==m0 and y0==n0)
(assume (y0>0) ;
  (assume (y0&1==1) ; 
    y1=y0-1 ; r1=r0+x0
  || assume (y0&1!=1) ;
    x0=x0 ; y1=y0 ; r1=r0) ;
  x1=x0<<1 ; y2=y1>>1
  (assume (y2>0) ;
    (assume (y2&1==1) ;
      y3=y2-1 ; r2=r1+x1
    || assume (y2&1!=1) ;
      x1=x1 ; y3=y2 ; r2=r1 ;
    x2=x1<<1 ; y4=y3>>1
    assume (y4<=0)
  || assume (y2<=0)) ;
    x2=x1 ; y4=y2 ; r2=r1)
|| assume(y0<=0) ; 
  x2=x0 ; y4=y0 ; r2=r0) ;
assert (r2==m0*n0)
```

IV. Vamos calcular o conjunto de fluxos sem escolha não-determinista:

```python
assume (m0>=0 and n0>=0 and r0==0 and x0==m0 and y0==n0)
assume (y0>0) ; assume (y0&1==1) ; y1=y0-1 ; r1=r0+x0 ; x1=x0<<1 ; y2=y1>>1
assume (y2>0) ; assume (y2&1==1) ; y3=y2-1 ; r2=r1+x1 ; x2=x1<<1 ; y4=y3>>1 ; assume (y4<=0)
assert (r2==m0*n0)

assume (m0>=0 and n0>=0 and r0==0 and x0==m0 and y0==n0)
assume (y0>0) ; assume (y0&1!=1) ; x0=x0 ; y1=y0 ; r1=r0 ; x1=x0<<1 ; y2=y1>>1
assume (y2>0) ; assume (y2&1==1) ; y3=y2-1 ; r2=r1+x1 ; x2=x1<<1 ; y4=y3>>1 ; assume (y4<=0)
assert (r2==m0*n0)

assume (m0>=0 and n0>=0 and r0==0 and x0==m0 and y0==n0)
assume (y0>0) ; assume (y0&1==1) ; y1=y0-1 ; r1=r0+x0 ; x1=x0<<1 ; y2=y1>>1
assume (y2>0) ; assume (y2&1!=1) ; x1=x1 ; y3=y2 ; r2=r1 ; x2=x1<<1 ; y4=y3>>1 ; assume (y4<=0)
assert (r2==m0*n0)

assume (m0>=0 and n0>=0 and r0==0 and x0==m0 and y0==n0)
assume (y0>0) ; assume (y0&1!=1) ; x0=x0 ; y1=y0 ; r1=r0 ; x1=x0<<1 ; y2=y1>>1
assume (y2>0) ; assume (y2&1!=1) ; x1=x1 ; y3=y2 ; r2=r1 ; x2=x1<<1 ; y4=y3>>1 ; assume (y4<=0)
assert (r2==m0*n0)

assume (m0>=0 and n0>=0 and r0==0 and x0==m0 and y0==n0)
assume (y0>0) ; assume (y0&1==1) ; y1=y0-1 ; r1=r0+x0 ; x1=x0<<1 ; y2=y1>>1
assume (y2<=0) ; x2=x1 ; y4=y2 ; r2=r1 ;
assert (r2==m0*n0)

assume (m0>=0 and n0>=0 and r0==0 and x0==m0 and y0==n0)
assume (y0>0) ; assume (y0&1!=1) ; x0=x0 ; y1=y0 ; r1=r0 ; x1=x0<<1 ; y2=y1>>1
assume (y2<=0) ; x2=x1 ; y4=y2 ; r2=r1 ;
assert (r2==m0*n0)

assume (m0>=0 and n0>=0 and r0==0 and x0==m0 and y0==n0)
assume (y0<=0) ; x2=x0 ; y4=y0 ; r2=r0 ;
assert (r2==m0*n0)
```

V. Para cada um destes fluxos, vamos calcular a respectiva denotação lógica (VC):

Variáveis:
```python
phi = m0>=0 and n0>=0 and r0==0 and x0==m0 and y0==n0
b1 = y0>0
i1 = y0&1==1
ci1 = y1==y0-1 and r1==r0+x0 and x1==x0<<1 and y2==y1>>1
b2 = y2>0
i2 = y2&1==1
ci2 = y3==y2-1 and r2==r1+x1 and x2==x1<<1 and y4==y3>>1 and y4<=0
psi = r1==m0*n0
cni1 = x0==x0 and y1==y0 and r1==r0 and x1==x0<<1 and y2==y1>>1
cni2 = x1==x1 and y3==y2 and r2==r1 and x2==x1<<1 and y4==y3>>1 and y4<=0
cnb2 = x2==x1 and y4==y2 and r2==r1
cnb1 = x2==x0 and y4==y0 and r1==r0
```

```python
assume (m0>=0 and n0>=0 and r0==0 and x0==m0 and y0==n0)
assume (y0>0) ; assume (y0&1==1) ; y1=y0-1 ; r1=r0+x0 ; x1=x0<<1 ; y2=y1>>1
assume (y2>0) ; assume (y2&1==1) ; y3=y2-1 ; r2=r1+x1 ; x2=x1<<1 ; y4=y3>>1 ; assume (y4<=0)
assert (r2==m0*n0)
```

($\phi \wedge$ b1 $\wedge$ i1 $\wedge$ ci1 $\wedge$ b2 $\wedge$ i2 $\wedge$ ci2) $\to \psi$ 

```python
assume (m0>=0 and n0>=0 and r0==0 and x0==m0 and y0==n0)
assume (y0>0) ; assume (y0&1!=1) ; x0=x0 ; y1=y0 ; r1=r0 ; x1=x0<<1 ; y2=y1>>1
assume (y2>0) ; assume (y2&1==1) ; y3=y2-1 ; r2=r1+x1 ; x2=x1<<1 ; y4=y3>>1 ; assume (y4<=0)
assert (r2==m0*n0)
```

($\phi \wedge$ b1 $\wedge \neg$ i1 $\wedge$ cni1 $\wedge$ b2 $\wedge$ i2 $\wedge$ ci2) $\to \psi$

```python
assume (m0>=0 and n0>=0 and r0==0 and x0==m0 and y0==n0)
assume (y0>0) ; assume (y0&1==1) ; y1=y0-1 ; r1=r0+x0 ; x1=x0<<1 ; y2=y1>>1
assume (y2>0) ; assume (y2&1!=1) ; x1=x1 ; y3=y2 ; r2=r1 ; x2=x1<<1 ; y4=y3>>1 ; assume (y4<=0)
assert (r2==m0*n0)
```

($\phi \wedge$ b1 $\wedge$ i1 $\wedge$ ci1 $\wedge$ b2 $\wedge \neg$ i2 $\wedge$ cni2) $\to \psi$

```python
assume (m0>=0 and n0>=0 and r0==0 and x0==m0 and y0==n0)
assume (y0>0) ; assume (y0&1!=1) ; x0=x0 ; y1=y0 ; r1=r0 ; x1=x0<<1 ; y2=y1>>1
assume (y2>0) ; assume (y2&1!=1) ; x1=x1 ; y3=y2 ; r2=r1 ; x2=x1<<1 ; y4=y3>>1 ; assume (y4<=0)
assert (r2==m0*n0)
```

($\phi \wedge$ b1 $\wedge \neg$ i1 $\wedge$ cni1 $\wedge$ b2 $\wedge \neg$ i2 $\wedge$ cni2) $\to \psi$

```python
assume (m0>=0 and n0>=0 and r0==0 and x0==m0 and y0==n0)
assume (y0>0) ; assume (y0&1==1) ; y1=y0-1 ; r1=r0+x0 ; x1=x0<<1 ; y2=y1>>1
assume (y2<=0) ; x2=x1 ; y4=y2 ; r2=r1 ;
assert (r2==m0*n0)
```

($\phi \wedge$ b1 $\wedge$ i1 $\wedge$ ci1 $\wedge \neg$ b2 $\wedge$ cnb2) $\to \psi$

```python
assume (m0>=0 and n0>=0 and r0==0 and x0==m0 and y0==n0)
assume (y0>0) ; assume (y0&1!=1) ; x0=x0 ; y1=y0 ; r1=r0 ; x1=x0<<1 ; y2=y1>>1
assume (y2<=0) ; x2=x1 ; y4=y2 ; r2=r1 ;
assert (r2==m0*n0)
```

($\phi \wedge$ b1 $\wedge \neg$ i1 $\wedge$ cni1 $\wedge \neg$ b2 $\wedge$ cnb2) $\to \psi$

```python
assume (m0>=0 and n0>=0 and r0==0 and x0==m0 and y0==n0)
assume (y0<=0) ; x2=x0 ; y4=y0 ; r2=r0 ;
assert (r2==m0*n0)
```

($\phi \wedge \neg$ b1 $\wedge$ cnb1) $\to \psi$

VI. Vamos provar as VCs que calculamos:

In [ ]:
m0, n0, r0, r1, r2, x0, x1, x2, y0, y1, y2, y3, y4 = BitVecs('m0 n0 r0 r1 r2 x0 x1 x2 y0 y1 y2 y3 y4', 16)

phi = And(m0>=0, n0>=0, r0==0, x0==m0, y0==n0)
b1 = y0>0
i1 = y0&1==1
ci1 = And(y1==y0-1, r1==r0+x0, x1==x0<<1, y2==y1>>1)
b2 = y2>0
i2 = y2&1==1
ci2 = And(y3==y2-1, r2==r1+x1, x2==x1<<1, y4==y3>>1, y4<=0)
psi = r1==m0*n0
cni1 = And(x0==x0, y1==y0, r1==r0, x1==x0<<1, y2==y1>>1)
cni2 = And(x1==x1, y3==y2, r2==r1, x2==x1<<1, y4==y3>>1, y4<=0)
cnb2 = And(x2==x1, y4==y2, r2==r1)
cnb1 = And(x2==x0, y4==y0, r1==r0)

f1 = Implies(And(phi,b1,i1,ci1,b2,i2,ci2),psi)

f2 = Implies(And(phi,b1,Not(i1),cni1,b2,i2,ci2),psi)

f3 = Implies(And(phi,b1,i1,ci1,b2,Not(i2),cni2),psi)

f4 = Implies(And(phi,b1,Not(i1),cni1,b2,Not(i2),cni2),psi)

f5 = Implies(And(phi,b1,i1,ci1,Not(b2),cnb2),psi)

f6 = Implies(And(phi,b1,Not(i1),cni1,Not(b2),cnb2),psi)

f7 =Implies(And(phi,Not(b1),cnb1),psi)

prove(f1)
prove(f2)
prove(f3)
prove(f4)
prove(f5)
prove(f6)
prove(f7)

A fórmula não é válida...
A fórmula não é válida...
A fórmula é válida!
A fórmula é válida!
A fórmula é válida!
A fórmula é válida!
A fórmula é válida!
